In [13]:
// https://www.hackerrank.com/challenges/dice-path
import scala.io.StdIn.{readInt, readLine}

case class DiceConfig(
  top: Short,
  back: Short,
  left: Short) {

  def opposing(x: Short): Short = (7-x).toShort

  def bottom = opposing(top)
  def front = opposing(back)
  def right = opposing(left)

  def rotateRight: DiceConfig = DiceConfig(top = left, back = back  , left = bottom)
  def rotateDown : DiceConfig = DiceConfig(top = back, back = bottom, left = left  )

}

case class Best(configs: Map[DiceConfig, Long]) extends AnyRef {

  def bestIfMovement(movement: DiceConfig => DiceConfig): Best = {
    Best(configs.map { case (config, bestSum) =>
      val newConfig = movement(config)
      val newSum = bestSum + newConfig.top
      (newConfig, newSum)
    })
  }

  def ifRotatedRight: Best = bestIfMovement(_.rotateRight)

  def ifRotatedDown: Best = bestIfMovement(_.rotateDown)

}

object Best {

  def max(best1: Best, best2: Best): Best = {
      val bigger = if(best1.configs.size > best2.configs.size) best1.configs else best2.configs
      val smaller = if(best1.configs == bigger) best2.configs else best1.configs
      var result = bigger
      smaller.foreach { 
          case (config,bestSum) =>
              result.get(config) match {
                  case Some(otherBest) if bestSum > otherBest => 
                      result = result.updated(config, bestSum)
                  case None =>
                      result = result.updated(config, bestSum)
                  case _ =>
                      ()
              }
      }
      Best(result)
  }

}

object DicePath {
  def solve(m: Short, n: Short): Long = {
    val initialDice = DiceConfig(top = 1, back = 5, left = 3)
    val bestSolutionAt11 = Best(configs = Map(initialDice -> initialDice.top))

    val solution =  if(m == 1) {
      (1 to n-1).toVector.foldLeft(bestSolutionAt11) {
        case (best,_) => best.ifRotatedRight
      }
    } else if(n == 1) {
      (1 to m-1).toVector.foldLeft(bestSolutionAt11) {
        case (best,_) => best.ifRotatedDown
      }
    } else {
      val firstRow = (1 to n-1).toVector.scanLeft(bestSolutionAt11) {
        case (best,_) => best.ifRotatedRight
      }.tail

      //println(s"firstRow = $firstRow")

      val firstColumn = (1 to m-1).toVector.scanLeft(bestSolutionAt11) {
        case (best,_) => best.ifRotatedDown
      }.tail

      //println(s"firstColumn = $firstColumn")

      firstColumn.foldLeft(firstRow) { (upperRow, leftmostSolution) =>
        upperRow.scanLeft(leftmostSolution) { (leftSolution, upperSolution) =>
          Best.max( upperSolution.ifRotatedDown, leftSolution.ifRotatedRight )
        }.tail
      }.last
    }

    solution.configs.values.max
  }
}

object Solution {

    def main(args: Array[String]) ={
        val testCases = readInt()
        for ( _ <- (1 to testCases)) {
        val spl = readLine().split("\\s+")
        val m = spl(0).toShort
        val n = spl(1).toShort
        println(DicePath.solve(m,n))
        }
    }
}

DicePath.solve(2,2)
DicePath.solve(1,2)
DicePath.solve(2,1)
DicePath.solve(3,3)

import scala.io.StdIn.{readInt, readLine}
defined class DiceConfig
defined class Best
defined object Best
defined object DicePath
defined object Solution
res12_6: Long = 9L
res12_7: Long = 4L
res12_8: Long = 6L
res12_9: Long = 19L